Reference: https://www.youtube.com/watch?v=xrdutbpfpho

In [1]:
import pandas as pd
import numpy as np
import vectorbt as vbt
import pandas_ta as ta

In [16]:
data = vbt.YFData.download(["BTC-USD","ETH-USD"],
                   start="2021-01-01").get("Close")
data

symbol,BTC-USD,ETH-USD
Date,,
2020-12-31 00:00:00+00:00,29001.720703,737.803406
2021-01-01 00:00:00+00:00,29374.152344,730.367554
2021-01-02 00:00:00+00:00,32127.267578,774.534973
2021-01-03 00:00:00+00:00,32782.023438,975.507690
2021-01-04 00:00:00+00:00,31971.914062,1040.233032
...,...,...
2023-08-15 00:00:00+00:00,29170.347656,1826.932739
2023-08-16 00:00:00+00:00,28701.779297,1805.659058
2023-08-17 00:00:00+00:00,26664.550781,1684.933472


In [28]:
grid = np.arange(10,250,10)
fast, slow = vbt.MA.run_combs(
    data,
    window = grid,
    r=2,
    short_names=["fast","slow"]
)
entries = fast.ma_crossed_above(slow)
exits = fast.ma_crossed_below(fast)

In [29]:
pf = vbt.Portfolio.from_signals(data, entries, exits, fees=0.001)
pf.total_return().unstack()

symbol                    BTC-USD   ETH-USD
slow_window fast_window                    
20          10          -0.293062 -0.081300
30          10          -0.335190 -0.053257
            20          -0.418844 -0.076968
40          10          -0.347322 -0.081300
            20          -0.418844 -0.074427
...                           ...       ...
240         190         -0.441466  0.107039
            200         -0.443168  0.031102
            210         -0.486229  0.046547
            220         -0.436267 -0.544422
            230         -0.400848 -0.474596

[276 rows x 2 columns]

### Combination strategies using Moving Average Cross Strategy & RSI Strategy

In [38]:
def get_signals(data, fast_window, slow_window, rsi_window, top, bottom):
    rsi = vbt.RSI.run(data, window=rsi_window, short_name = "RSI").rsi.to_numpy()
    fast = vbt.MA.run(data, window=fast_window, short_name="fast").ma.to_numpy()
    slow = vbt.MA.run(data, window=slow_window, short_name="slow").ma.to_numpy()
    entries = (fast > slow) & (rsi < top)
    exits = (fast < slow) & (rsi > bottom)
    return entries, exits

entries, exits = get_signals(data, 180,240, 14, 80,20)
pf = vbt.Portfolio.from_signals(data, entries, exits, fees=0.001)
pf.total_return()  

symbol
BTC-USD   -0.328086
ETH-USD   -0.255416
Name: total_return, dtype: float64

In [39]:
def combine_rsi_ma(data, fast_window, slow_window, rsi_window, top, bottom):
    rsi = vbt.RSI.run(data, window=rsi_window).rsi.to_numpy()
    fast = vbt.MA.run(data, window=fast_window).ma.to_numpy()
    slow = vbt.MA.run(data, window=slow_window).ma.to_numpy()
    indicator = np.where((fast > slow) & (rsi < top), 1, 0)
    indicator = np.where((fast < slow) & (rsi > bottom), -1, indicator)
    return indicator

indicator_rsi_ma = vbt.IndicatorFactory(
    class_name='Combination_RSI_MA',
    short_name='RSI_MA',
    input_names=['close'],
    param_names=['fast_window', 'slow_window', 'rsi_window', 'top', 'bottom'],
    output_names=['value'],
).from_apply_func(combine_rsi_ma, fast_window=180, slow_window=240, rsi_window=14, top=70, bottom=30)


res = indicator_rsi_ma.run(
    data,
    fast_window=180,
    slow_window=240,
    rsi_window=14,
    top=80,
    bottom=20
)


#entries = res.value_crossed_above(0)
#exits = res.value_crossed_below(0)
entries = res.value_above(0)
exits = res.value_below(0)

pf = vbt.Portfolio.from_signals(data, entries, exits, fees=0.001)
pf.total_return()

RSI_MA_fast_window  RSI_MA_slow_window  RSI_MA_rsi_window  RSI_MA_top  RSI_MA_bottom  symbol 
180                 240                 14                 80          20             BTC-USD   -0.328086
                                                                                      ETH-USD   -0.255416
Name: total_return, dtype: float64